This notebook contains tasks related to probability and their solutions using the Monte Carlo method.

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
from random import shuffle
from tqdm.notebook import tqdm

#Tasks

## TASK 1
The exam is conducted as follows: if a particular ticket has already been drawn, the examiner sets it aside after the student has answered the question. The student has memorized 20 of the 30 tickets. When is it more advantageous for him to go first or second, so that the probability of drawing a memorized ticket is higher?

In [ ]:
tickets = [i for i in range(1, 31)]
student = [i for i in range(1, 21)]
n = 10000
result = []

for _ in tqdm(range(n)):
  shuffle(tickets)
  result.append(tickets[0] in student)

print(np.mean(result))

  0%|          | 0/10000 [00:00<?, ?it/s]

0.6677


## TASK 2
At night, there was a traffic accident involving a taxi. There are two taxi companies operating in the city: “Blue” and “Green.” ‘Green’ owns 85% of the taxis, and “Blue” owns 15%. A witness to the accident claims that the taxi belonged to Blue. An investigative experiment showed that at night, the witness correctly identifies the color of a taxi 80% of the time. What is the probability that the taxi really belonged to Blue?

In [ ]:
result = []
n = 10000

def witness(taxi):
  if np.random.binomial(1, 0.8):
    return taxi
  return np.abs(taxi-1)


for _ in tqdm(range(n)):
  taxi = np.random.binomial(1, 0.15)
  witness_predict = witness(taxi)
  result.append((taxi, witness_predict))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(result, columns=['taxi', 'witness_predict'])

df.groupby('witness_predict')['taxi'].mean()

,taxi
witness_predict,
0,0.043922
1,0.402107


In [ ]:
df[(df['witness_predict'] == 1) & (df['taxi'] == 1)].shape[0] / df[df['witness_predict'] == 1].shape[0]

0.4021074099252209

## TASK 3
Russian roulette. There is a revolver with two bullets loaded in a row. The first person spins the cylinder and shoots, and survives. It is your turn: you are offered to spin the cylinder before shooting. Will you agree or shoot immediately, and why?

In [ ]:
def one_turn(chamber):
  new_chamber = [0, 0, 0, 0, 0, 0]
  n = len(chamber)
  for i in range(n):
    if i < n-1:
      new_chamber[i+1] = chamber[i]
    else:
      new_chamber[0] = chamber[i]
  return new_chamber

def spin_chamber(chamber):
  n = np.random.randint(1, 7)
  for _ in range(n):
    chamber = one_turn(chamber)
  return chamber

In [ ]:
pd.Series([spin_chamber([1, 1, 0, 0, 0, 0]) for _ in range(100000)]).astype(str).value_counts(normalize=True)

,proportion
"[0, 0, 0, 1, 1, 0]",0.16813
"[1, 1, 0, 0, 0, 0]",0.16772
"[1, 0, 0, 0, 0, 1]",0.16690
"[0, 0, 1, 1, 0, 0]",0.16652
"[0, 0, 0, 0, 1, 1]",0.16645
"[0, 1, 1, 0, 0, 0]",0.16428


In [ ]:
n = 10000
result = []

for _ in range(n):
  chamber = [1,1,0,0,0,0]
  # first player
  chamber = spin_chamber(chamber)
  chamber = one_turn(chamber)
  p1 = chamber[-1]
  chamber[-1] = 0
  # second player
  chamber = spin_chamber(chamber)
  chamber = one_turn(chamber)
  p2 = chamber[-1]

  result.append((p1,p2))

In [ ]:
df = pd.DataFrame(result, columns=['p1', 'p2'])
df.groupby('p1')['p2'].agg(['count','mean'])

,count,mean
p1,,
0,6680,0.335928
1,3320,0.165964


In [ ]:
n = 10000
result = []

for _ in range(n):
  chamber = [1,1,0,0,0,0]
  # first player
  chamber = spin_chamber(chamber)
  chamber = one_turn(chamber)
  p1 = chamber[-1]
  chamber[-1] = 0
  # second player
  # chamber = spin_chamber(chamber)
  chamber = one_turn(chamber)
  p2 = chamber[-1]

  result.append((p1,p2))

In [ ]:
df = pd.DataFrame(result, columns=['p1', 'p2'])
df.groupby('p1')['p2'].agg(['count','mean'])

,count,mean
p1,,
0,6630,0.248718
1,3370,0.511276


We can see that spinning the cylinder isn't necessary to decrease the probability of death.

## TASK 4
Determine the probability that in a group of 23 people, two people will have the same birthday (day and month).

In [ ]:
birth_series = pd.Series(np.arange(1, 366))

result = [any(birth_series.sample(23, replace=True).duplicated()) for _ in range(1000)]
np.mean(result)

np.float64(0.522)